In [1]:
import numpy as np
import os
import time
import argparse
import random
import sys

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, Flatten, BatchNormalization, Activation

from tensorflow.keras.optimizers import Adam
from qkeras import QActivation, QDense, QConv1D, QConv2D, quantized_bits
from qkeras.autoqkeras.utils import print_qmodel_summary

from sklearn.metrics import accuracy_score
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from node_edge_projection import NodeEdgeProjection

sys.path.append("..")
import util.data
import util.plots
import util.util
from util.terminal_colors import tcols

os.environ['CUDA_VISIBLE_DEVICES'] = "0"


2023-09-19 19:59:53.647179: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# replace with your own model_dir
model_dir = "nconst8_nbits8_De3_Do6_NL0_SE0_SN6_SG6144_batch512_acc0_P0_Seed1_Kfold_20230919-195709"

nmax = 8
# Quantized bits

nbits = 8
integ = 0

In [3]:

fpath = f'./data_kfold/jets_{nmax}constituents_ptetaphi_nonorm'
fname_test = f'jet_images_c{nmax}_minpt2.0_ptetaphi_nonorm'

data = util.data.Data(fpath=fpath, fname=fname_test, only_test=True)

X_test = data.test_data
Y_test = data.test_target

interquantile_range_32 = [120, 0.27, 0.27]
interquantile_range_16 = [166, 0.24, 0.24]
interquantile_range_8  = [219, 0.20, 0.20]
    
if nmax == 8:
    X_test      = X_test      / interquantile_range_8
elif nmax == 16:
    X_test      = X_test      / interquantile_range_16
elif nmax == 32:
    X_test      = X_test      / interquantile_range_32
    
    # The dataset is N_jets x N_constituents x N_features
njet = X_test.shape[0]
nconstit = X_test.shape[1]
nfeat = X_test.shape[2]

ntargets = Y_test.shape[1]
    
print("#jets = ", njet)
print("#constituents = ", nconstit)
print("#targets = ", ntargets)
print("#features = ", nfeat)
    
print(X_test.shape, Y_test.shape)


----------------
Data loading complete:
File name: jet_images_c8_minpt2.0_ptetaphi_nonorm
Training data size: 0
Test data size: 252,340
Number of constituents: 8
Number of features: 3
----------------

#jets =  252340
#constituents =  8
#targets =  5
#features =  3
(252340, 8, 3) (252340, 5)


In [4]:

kfolds = 5
accuracy_keras = []
kfold_metrics = {
    "fprs": [],
    "aucs": [],
    "fats": [],
    "accs": [],
    "loss": []
}

for i in range (kfolds):
    
    val_kfold = i
    #train_kfolds = [kfold for kfold in range(kfolds) if kfold != test_kfold]

    mname = "model_QInteractionNetwork_nconst_{}_nbits_{}_kfold_{}".format(
            nmax, 
            nbits,
            val_kfold,
    )
        
    model = tf.keras.models.load_model(
        "{}/{}.h5".format(model_dir, mname),
        custom_objects={
            "QDense": QDense,
            "QActivation": QActivation,
            "QConv1D": QConv1D,
            "QConv2D": QConv2D,
            "quantized_bits": quantized_bits,
            #"GarNet": GarNet,
            "NodeEdgeProjection": NodeEdgeProjection,
            "PruneLowMagnitude": pruning_wrapper.PruneLowMagnitude,
        },
    )
    
    y_keras = model.predict(X_test)
    
    accuracy_keras.append(float(
        accuracy_score(np.argmax(Y_test, axis=1), np.argmax(y_keras, axis=1))
    ))
    
    acc = keras.metrics.CategoricalAccuracy()
    acc.update_state(Y_test, y_keras)
    
    categorical_crossent = keras.losses.CategoricalCrossentropy()
    ce_loss = categorical_crossent(Y_test, y_keras).numpy()
    
    plots_dir = model_dir
    roc_metrics = util.plots.roc_curves(plots_dir, y_keras, Y_test, val_kfold)
    util.plots.dnn_output(plots_dir, y_keras)
    
    model_kfold_metrics = {}
    model_kfold_metrics.update({"fprs": roc_metrics[0]})
    model_kfold_metrics.update({"aucs": roc_metrics[2]})
    model_kfold_metrics.update({"fats": roc_metrics[3]})
    model_kfold_metrics.update({"accs": acc.result().numpy()})
    model_kfold_metrics.update({"loss": ce_loss})

    for key in model_kfold_metrics.keys():
        kfold_metrics[key].append(model_kfold_metrics[key])
        
    
print("inference done")    

2023-09-19 20:00:36.169393: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-09-19 20:00:36.222448: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-19 20:00:36.222707: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2023-09-19 20:00:36.222722: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-09-19 20:00:36.241004: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-09-19 20:00:36.241056: I tensorflow/stream_executor/pl

ROC curves plot saved to nconst8_nbits8_De3_Do6_NL0_SE0_SN6_SG6144_batch512_acc0_P0_Seed1_Kfold_20230919-195709.
DNN output histograms plot saved to nconst8_nbits8_De3_Do6_NL0_SE0_SN6_SG6144_batch512_acc0_P0_Seed1_Kfold_20230919-195709.
ROC curves plot saved to nconst8_nbits8_De3_Do6_NL0_SE0_SN6_SG6144_batch512_acc0_P0_Seed1_Kfold_20230919-195709.
DNN output histograms plot saved to nconst8_nbits8_De3_Do6_NL0_SE0_SN6_SG6144_batch512_acc0_P0_Seed1_Kfold_20230919-195709.
ROC curves plot saved to nconst8_nbits8_De3_Do6_NL0_SE0_SN6_SG6144_batch512_acc0_P0_Seed1_Kfold_20230919-195709.
DNN output histograms plot saved to nconst8_nbits8_De3_Do6_NL0_SE0_SN6_SG6144_batch512_acc0_P0_Seed1_Kfold_20230919-195709.
ROC curves plot saved to nconst8_nbits8_De3_Do6_NL0_SE0_SN6_SG6144_batch512_acc0_P0_Seed1_Kfold_20230919-195709.
DNN output histograms plot saved to nconst8_nbits8_De3_Do6_NL0_SE0_SN6_SG6144_batch512_acc0_P0_Seed1_Kfold_20230919-195709.
ROC curves plot saved to nconst8_nbits8_De3_Do6_NL0_

In [8]:
print(kfold_metrics)

accuracy_average = np.mean(np.array(accuracy_keras))
accuracy_errs = np.std(np.array(accuracy_keras))
print(accuracy_average)
print(accuracy_errs)
print(accuracy_keras)



{'fprs': [[array([0.00222914, 0.00318023, 0.00425022, 0.00538955, 0.00651403,
       0.00783665, 0.00909983, 0.01024907, 0.0115172 , 0.01285468,
       0.01444975, 0.01602996, 0.01747147, 0.01897669, 0.02061702,
       0.02236566, 0.02395577, 0.02559047, 0.02734406, 0.02914223,
       0.03092554, 0.03307046, 0.03534913, 0.03763771, 0.03965384,
       0.04174427, 0.04387434, 0.04615796, 0.04849608, 0.05116609,
       0.05350304, 0.0559612 , 0.05879468, 0.06143992, 0.06437247,
       0.06749822, 0.07070817, 0.07358128, 0.07657327, 0.07976341,
       0.08267615, 0.08622791, 0.08989855, 0.09392585, 0.09788381,
       0.1015594 , 0.10566597, 0.1097973 , 0.11399798, 0.11852065,
       0.12286003, 0.12783843, 0.1326385 , 0.13752774, 0.14294702,
       0.14847527, 0.15380043, 0.15916026, 0.16497087, 0.1708657 ,
       0.17688436, 0.18270488, 0.1894121 , 0.19669394, 0.20409467,
       0.21092078, 0.21906951, 0.22686158, 0.23573849, 0.24405069,
       0.25283843, 0.26248811, 0.27151859, 0.280573

In [9]:
tprs_baseline = roc_metrics[1]


avg_metrics = {}
for key, value in kfold_metrics.items():
    avg_metrics.update(
        {f"{key}": np.mean(value, axis=0), f"{key}_errs": np.std(value, axis=0)}
    )
variance_fats = np.var(kfold_metrics['fats'], axis=0)

roc_uncert_plot_data = avg_metrics.copy()
del roc_uncert_plot_data['accs']
del roc_uncert_plot_data['accs_errs']
del roc_uncert_plot_data['loss']
del roc_uncert_plot_data['loss_errs']


In [21]:

roc_uncert_plot_data.update({'tpr': tprs_baseline, 'outdir': model_dir})
util.plots.roc_curves_uncert(**roc_uncert_plot_data)

inverse_fats = 1 / np.mean(avg_metrics['fats'])
inverse_fats_error = inverse_fats**2 * np.sqrt(np.sum(variance_fats))

print(f"Accuracy: {avg_metrics['accs']:.3f} \u00B1 {avg_metrics['accs_errs']:.3f}")
print(f"Avg loss: {avg_metrics['loss']:.3f} \u00B1 {avg_metrics['loss_errs']:.3f}")
print(f"Average 1/<FPR>: {inverse_fats:.3f} \u00B1 {inverse_fats_error:.3f}")


    
    

ROC curves plot saved to nconst8_nbits8_De3_Do6_NL0_SE0_SN6_SG6144_batch512_acc0_P0_Seed1_TK5_20230916-145549.
Accuracy: 0.593 ± 0.003
Avg loss: 1.056 ± 0.005
Average 1/<FPR>: 3.968 ± 1.221
